# PRÁCTICA 1 BaseV2 - ALGORITMO BASADOS EN ENTORNOS Y TRAYECTORIAS

## Carga de datos

In [68]:
import numpy as np
import pandas as pd

index_df = pd.read_csv('./bicicletas/cercanas_indices.csv')
kms_df = pd.read_csv('./bicicletas/cercanas_kms.csv')
deltas_df = pd.read_csv('./bicicletas/deltas_5m_double.csv')

In [20]:
def movements_to_list(deltas_df):
    """
    Devuelve array con todos los movimientos realizados en las estaciones. Cada posicion del array esta compuesto por el id de la estacion y la cantidad de bicis retiradas/aniadidas en cada movimiento
    
    Parametros
        deltas_df : DataFrame
            Pandas DataFrame a convertir
    Return
        Numpy array
    """
    move_list = np.array([]) # Inicializamos el array
    rows = len(deltas_df.index) # Mediciones totales realizadas
    columns = len(deltas_df.columns) # Numero de estaciones
    
    # Recorreremos todas las mediciones e iremos transformando todas ellas que sean distintas de 0
    for i in range(0, rows):
        if i != 0:
            row = deltas_df.iloc[i] # Seleccionamos una medicion
            selec = row[row != 0] # Seleccionamos que estaciones han sufrido algun cambio
            for j in range(0, len(selec)):
                lst = np.array([[int(selec.index[j]), selec[j]]]) # Obtenemos un array con el index y el numero de bicis retiradas/aniadidas
                # En la setencia anterior, el primer corchete sirve para crear el array. Mientras que el segundo para crear una tupla tal que: [estacion, movements]
                if len(move_list) == 0:
                    move_list = lst # Asignamos la primera tupla como 
                else:
                    move_list = np.append(move_list, lst, axis=0) # Aniadimos una nueva fila(axis=0) a move_list
    
    return move_list

## Función de evaluación

In [21]:
def evaluate(move_list, init_state, capacity, index_m, kms_m):
    '''
    La funcion estipula el numero de kilometros recorrido por los usuarios al no encontrar una bicicleta en su estación de preferencia y tener que desplazarse a la estaion mas cercana con bicicletas disponibles, o la distancia recorrida en caso de no haber sitio para dejar la bicicleta en la estación deseada y tener que desplazarse a la estacion mas cercana con capacidad. La distancia realizada a pie es 3 veces mayor que la recorrida en bicicleta
    
    Parametros
        move_list : Numpy ndarray
            lista con todos los movimientos realizados. Cada elemento de la lista debe tener la siguiente estructura: [index, desplazamientos]
        init_state : Numpy ndarray
            Contiene el numero de bicicletas alojadas en cada estacion al comienzo de la evaluacion
        capacity : Numpy ndarray
            Contiene la capacidad maxima de bicicletas que puede almacenar una estacion
        index_m : Numpy ndarray
            Almacena las estaciones mas cercanas a cada estacion ordenadas de mas cercanas a mas lejanas. La fila indica la estacion de refencia, mientras que la columna como de cerca esta la estacion
        kms_m : Numpy ndarray
            Almacena la distancia que hay desde una estacion a otra. La disposicion de la informacion se basa en la variable index_m. Es decir, el valor de cada celda de esta matriz corresponde a la posicion homologa en index_m
    Returns
        El numero total de kilometros recorridos por los usuarios
    '''
    tkms = 0 # Kilometros totales
    travel_kms = 0 # Kilometros recorridos por un usuario
    actual_state = init_state.copy() # Variable en la que iremos modificando el numero de biciclestas que se encuentran en cada estacion
    walk_multiplier = 3 # La distancia recorrida andando costara 3 veces mas que la recorrida en bici (valor calculado en kms_m es la distancia en bici)
    
    for move in move_list:
        station = move[0] # Guardamos la estacion 
        n_bicycle = actual_state[station] + move[1]  # Al numero de bicis de la estacion es cuestion, se le suma el numero de bicis desplazadas (este valor puede ser positivo o negativo) 
        
        # Tenemos que comprobar que el numero de desplazamientos es posible. Es decir, en caso de que se hayan retirado biciletas, que hubiese suficientes para suplir la demanda.
        # Y caso de que se quisiera dejar bicicletas, que existan suficientes slots/espaciones disponibles en la estacion
        if n_bicycle >= 0 and n_bicycle <= capacity[station]: 
            actual_state[station] = n_bicycle # Como el numero obtenido es posible, actualizamos el estado actual de la estacion
        else:
            # En caso de que no existan bicicletas suficientes para suplir la demanda en la estacion
            if n_bicycle < 0:
                actual_state[station] = 0 # La estacion ahora tiene 0 bicicletas
                search = abs(n_bicycle) # abs() nos devuelve el valor absoluto. Obtenemos el valor de bicicletas que necesitamos buscar
                
                # Ahora debemos buscar la estacion mas cercana con capacidad suficiente. Pero podria pasar que una unica estacion no tuviera todos los slots necesarios
                nearest_station_index = find_nearest_station_index(index_m, station, actual_state, capacity, search, 'bicycles') # Obtenemos la estacion mas cercana con capacidad suficiente de bicicletas

                for near_station in nearest_stations_index:
                    index = near_station[0]
                    bicycles = near_station[1]

                    # Calculamos los kms recorridos para llegar a la estacion mas cercana
                    travel_kms = kms_m[station][index] * bicycles * walk_multiplier # Kms hacia la estacion * numero de bicicletas a buscar * multiplicador por andar

                    # Actualizamos el estado de la estacion mas cercana
                    nearest_station = index_m[station][index] # Estacion mas cercana a la nuestra
                    actual_state[nearest_station] -= bicycles # Le restamos el numero de bicicletas necesario
                    tkms += travel_kms # Sumamos los kilometros recorridos al total
                        
            # En caso de que no existan slots suficientes para suplir la demanda en la estacion
            elif n_bicycle > capacity[station]:
                actual_state[station] = capacity[station] # Ahora la estacion esta llena
                search = n_bicycle - capacity[station] # Obtenemos el numero de slots a buscar
                
                # Ahora debemos buscar la estacion mas cercana con capacidad suficiente. Pero podria pasar que una unica estacion no tuviera todos los slots necesarios
                nearest_stations_index = find_nearest_station_index(index_m, station, actual_state, capacity, search, 'slots') # Obtenemos la estacion/estaciones mas cercanas con capacidad suficiente de slots
                
                for near_station in nearest_stations_index:
                    index = near_station[0]
                    slots = near_station[1]

                    # Calculamos los kms recorridos para llegar a la estacion mas cercana
                    travel_kms = kms_m[station][index] * slots # Kms hacia la estacion * numero de slots a buscar

                    # Actualizamos el estado de la estacion mas cercana
                    nearest_station = index_m[station][index] # Estacion mas cercana a la nuestra
                    actual_state[nearest_station] += slots # Le sumamos el numero de bicicletas dejas en los slots libres
                    tkms += travel_kms # Sumamos los kilometros recorridos al total


    return tkms


## Encontrar estación más cercana

In [22]:
def find_nearest_station_index(index_m, station, actual_state, capacity, search, search_type):
    """
    Devuelve una lista con los siguientes valores:
        * index-columna: Index de la estacion mas cercana con respecto al parametro station en la variable index_m
        * slots/bicicletas: Numero de slots/bicicletas aniadidas/quitadas en la estacion index-columna
    
    Parametros
        index_m : Numpy ndarray
            Almacena las estaciones mas cercanas a cada estacion ordenadas de mas cercanas a mas lejanas. La fila indica la estacion de refencia, mientras que la columna como de cerca esta la estacion
        station : int
            Estacion desde la que buscar su estacion vecina mas proxima
        actual_state : Numpy ndarray
            Array con el estado actual de bicicletas en cada estacion
        capacity : Numpy ndarray
            Contiene la capacidad maxima de bicicletas que puede almacenar una estacion
        search : int
            Numero de bicicletas/slots a buscar
        search_type : str
            Cadena de caracteres que nos indica el tipo de elemento a buscar: 'bicycles' o 'slots'
            
    Return
        Devuelve una lista con los indices de las estaciones en la matriz index_m y el numero de biciletas/slots cogidos en cada una de las estaciones. Ejemplo:
        [[station_index=3, slots=2], [station_index=12, bicicletas=4]]
    """
    # Valor que almacena la posicion (columna) de la estacion mas cercana con respecto al parametro station
    row_selec = index_m[station] # Seleccionamos el array de estaciones cercanas de la estacion station
    index_lst = np.array([]) # Lista donde guardaremos el index de la estaciones mas cercanas, junto al numero de bicicletas/slots asignados
    remaining = search # Numero de slots que aun faltan por asignar
    
    # Comprobamos que tipo de busqueda estamos realizando, 'bicycles' o 'slots'
    if search_type == 'bicycles':
        # Recorremos el array seleccionado y comprobamos cual es la estacion mas cercana con bicicletas suficientes
        for j in range(1, len(row_selec)):
            # Recorremos el array seleccionado y comprobamos cuales son las estaciones mas cercanas con las bicicletas suficientes
            free_bicycles = actual_state[row_selec[j]]
            
            # Si la estacion tiene biciclestas disponibles se les asigna, en caso contrario comprobamos la siguiente estacion
            if free_bicycles > 0:
                # Cuando el numero de bicicletas de la estacion no suple la demanda
                if remaining > free_bicycles:
                    remaining = remaining - free_bicycles # Actualizamos el numero de bicicletas que faltan por asignar
                    taken = free_bicycles # Debemos tomar estas bicis
                # Cuando el numero de bicicletas de la estacion suple la demanda
                else:
                    taken = remaining # Del numero de bicicletas total disponible. Solo tomamos los necesarios
                    remaining = 0              
                
                index = np.array([j, taken]) # Creamos la variable que contiene la estacion y el numero 
                
                # La primera vez que guardamos un index debemos de asignarlo a index_lst
                if len(index_lst) == 0:
                    index_lst = index
                else:
                    index_lst = np.append(index_lst, index, axis=0) # Aniadimos una nueva fila(axis=0) a move_list
                    
                # El bucle termina cuando hemos asignado todos los slots
                if remaining == 0:
                    return index_lst # Devolvemos la lista de movimientos 
            
        raise Exception("Error, ninguna estacion tiene bicicletas")
            # Seleccionamos con row_selec la estacion mas cercana a station, despues comprobamos la capacidad disponibles. Y por ultimo comprobamos si es suficiente
            #if search <= actual_state[row_selec[j]]:
            #    return j # Devolvemos el index
            
    elif search_type == 'slots':
        # Recorremos el array seleccionado y comprobamos cuales son las estaciones mas cercanas con los slots suficientes
        for j in range(1, len(row_selec)):
            # Seleccionamos con row_selec la estacion mas cercana a station, despues comprobamos la capacidad disponibles. Y por ultimo comprobamos si es suficiente
            free_slots = capacity[row_selec[j]] - actual_state[row_selec[j]] # Calculamos el numero de slots disponibles restandole a la capacidad total el numero de bicicletas actuales
            
            # Si la estacion tiene slots disponibles se les asigna, en caso contrario comprobamos la siguiente estacion
            if free_slots > 0:
                # Cuando el numero de slots de la estacion no suple la demanda o es igual que la demanda
                if remaining > free_slots:
                    remaining = remaining - free_slots # Actualizamos los slots que faltan por asignar
                    taken = free_slots # Debemos tomar estos slots

                # Cuando el numero de slots de la estacion si suple la demanda
                else:
                    taken = remaining # Del numero de slots total disponible. Solo tomamos los necesarios
                    remaining = 0
                
                index = np.array([[j, taken]])

                # La primera vez que guardamos un index debemos de asignarlo a index_lst
                if len(index_lst) == 0:
                    index_lst = index
                else:
                    index_lst = np.append(index_lst, index, axis=0) # Aniadimos una nueva fila(axis=0) a move_list

                # El bucle termina cuando hemos asignado todos los slots
                if remaining == 0:
                    return index_lst # Devolvemos la lista de movimientos    
        
        raise Exception("Error, todas las estaciones estan llenas")
    else:
        raise Exception("La variable search_type no contiene un tipo valido") # La palabra 'raise' nos permite lanzar una excepcion. Es igual que la palabra 'throw' en otros lenguajes

## Mostrar variación (Función no necesaria)

In [23]:
def show_variation(init_solution, init_tkms, new_solution, new_tkms):
    """
    Muestra por pantalla la diferencia de slots existentes entre dos soluciones

    Parametros
        init_solution : Numpy ndarray
            Solucion inicial a comparar
        init_tkms : float
            Numero de kilometros recorridos en la solucion inicial
        new_solution : Numpy ndarray
            Solucion a comparar con respecto a init_solution
        new_tkms : float
            Numero de kilometros recorridos en la nueva solucion
    """

    columns = np.arange(0,len(init_solution))
    variation = init_solution - new_solution

    # Para representarlo carrectamente la variacion crearemos un string con los espacios correspondientes
    string = '['
    space = ' '
    for i in range(0,len(init_solution)):
        # Es caso de que la variacion sea mayor que 0 y menor a 10, sera necesario aniadir un espacio
        if 10 > variation[i] and variation[i] >= 0:
            if variation [i] == 0: # Cuando aniadimos un 0 debemos agregar un espacio delante. Cuando es un numero positivo, un signo +
                string += space + str(variation[i])
            else:
                string += '+' + str(variation[i])
        else:
            string += str(variation[i])


        if i != len(init_solution)-1:
            string += space

    string += ']'

    print('')
    print('----------------------COMPARACION DE SOLUCIONES-----------------------')
    print('    Estacion    : {}'.format(columns))
    print('----------------------------------------------------------------------')
    print('Solucion inicial: {} - {} km'.format(init_solution, init_tkms))
    print(' Solucion final : {} - {} km'.format(new_solution, new_tkms))
    print('----------------------------------------------------------------------')
    print('    Variacion   : ' + string)
    print('----------------------------------------------------------------------')
    print('')

## Generación de la solución inicial

### GREEDY

Deberemos aumentar el número de cada estación de forma proporcional al número que ya tienen en este momento. Para ello, calcularemos el número total de bicicletas existentes en el estado inicial y dividiremos cada estación por este número. Después aumentaremos estos valores para que la suma total sea igual a 220

In [83]:
def generate_initial_greedy_solution(init_state, max_slots):
    '''
    Crea una solucion inicial para el algoritmo greedy a partir del numero maximo de slots disponibles y el estado inicial
    
    Parametros
        init_state : Numpy ndarray
            Estado inicial de bicletas en las estaciones
        max_slots : int
            Numero maximo de slots disponibles a repartir

    Return
        Devuelve un array con el numero de slots disponibles en cada estacion (capacity)
    '''
    stations = len(init_state)
    solution = init_state.copy() # Array donde guardaremos la solucion, cada valor del vector representa el numero de slots que tiene
    proportion = max_slots / init_state.sum() # Nueva proporcion para el maximo de slots
    
    # station * proportion
    solution = np.around(solution * proportion).astype(int) # np.around() nos permite redondear los valores del array. Despues le hacemos cast a int
    print(solution)
    return solution
    

### EL RESTO DE ALGORITMOS

Crearemos un array con el tamaño de las estaciones, con valor 0. Recorreremos el array y le asignaremos un valor de slots a cada una de forma aleatoria. Para ello, tendremos un rango máximo y mínimo. De forma que, una vez calculado dicho número se hará lo mismo con la siguiente estación. Cuando se llegue a la última estación se le asignara el número restante de slots a repartir

In [25]:
def generate_initial_solution(init_state, max_slots):
    '''
    Crea una solucion a partir del estado inicial y el numero maximo de slots disponibles
    
    Parametros
        init_state : Numpy ndarray
            Estado inicial
        max_slots : int
            Numero maximo de slots disponibles a repartir

    Return
        Devuelve un array con el numero de slots disponibles en cada estacion (capacity)
    '''
    n_stations = len(init_state)
    
    # 2 3 4 3 2 4 5 2 5
    # 2 3 4 3 2 9 4 1 3
    
    # Usaremos un metodo parecido al usado en la generacion greedy. Generaremos N numeros aleatorios entre min_value y max_value, multiplicamos
    # cada uno por la proporcion entre max_slots/(la suma de todos ellos) y redondear por defecto. 
    
    max_value = round(max_slots/n_stations)
    min_value = 7
    margin = 20 # Margen para hacer que la solucion generada sea compatible con el estado actual
    accept = False # Varible de control
    
    while accept == False:
        solution = np.random.randint(min_value, max_value, n_stations) # Crea un array de numeros aleatorios con el tamanio del numero de estaciones
        # Haremos que la proporcion no este correlacionada exactammente a max_slots, estableceremos un valor mas pequenio para luego poder asignar valores
        # a los las estaciones que no sean compatibles con la solucion inicial
        proportion = (max_slots-margin) / solution.sum() # Nueva proporcion para el maximo de slots
        
        # station * proportion
        solution = np.around(solution * proportion).astype(int) # np.around() nos permite redondear los valores del array. Despues le hacemos cast a int
        amount = solution.sum() 
        remaining = max_slots - amount # Slots restantes por asignar
        
        # Comprobamos a que estacion le faltan slots para satisfacer el estado inicial
        lst = solution < init_state
        for i in range(0,len(lst)):
            if lst[i] == True:
                rest = init_state[i] - solution[i]
                if (remaining - rest) >= 0: 
                    solution[i] += rest # Aniadimos el numero de slots necesarios
                    remaining -= rest # Le quitamos los aniadidos al total restante
                else:
                    break # Si no hay slots suficientes
        
        # Si faltan slots por asignar recorreremos el array asignando slots
        if remaining >= 0:
            # Ahora que tenemos una solucion aceptable aniadimos los slots restantes. Vamos aniadiendo a cada estacion los valores 1 a 1
            i = 0
            while remaining > 0:
                solution[i] += 1 
                remaining -= 1
                i += 1
                if i == n_stations:
                    i = 0
            
            check = solution > init_state
            # Si todas los estaciones de la solucion tienen los slots suficientes para albergar a la solucion inicial. Entonces se acepta
            if check.sum() == n_stations:
                accept = True 

    return solution.astype(int)

In [26]:
# ESTA VERSION NO GENERA UNA SOLUCION COMPATIBLE CON EL ESTADO INICIAL
def generate_solution(n_stations, max_slots):
    '''
    Crea una solucion a partir del numero maximo de slots disponibles. No tiene por que ser compatible con el estado inicial
    
    Parametros
        n_stations : int
            Numero de estaciones que tiene el problema
        max_slots : int
            Numero maximo de slots disponibles a repartir

    Return
        Devuelve un array con el numero de slots disponibles en cada estacion (capacity)
    '''
    # np.random.seed(seed) # Inicializamos la semilla
    solution = np.zeros(16) # Array donde guardaremos la solucion, cada valor del vector representa el numero de slots que tiene
    
    # Debemos establecer un rango de slots que podemos aniadir en cada estacion
    max_value = 30 # Numero minimo de slots que se le puede dar a una estacion
    min_value = 5 # Numero minimo de slots que se le puede dar a una estacion
    assign_slots = 0 # Numero de slots asignados en cada momento
    remaining = max_slots # Slots que faltan por asignar

    for i in range(0,n_stations):
        # Comprobamos el numero de slots que quedan por asignar
        remaining = remaining - assign_slots
        
        # En la estacion debemos asignar todos los slots restantes
        if i != (n_stations-1):
            # Comprobamos que el valor maximo a asignar, es menor que los slots que quedan por repartir
            if remaining < max_value:
                max_value = remaining
                min_value = 0
                
            assign_slots = np.random.randint(min_value, max_value) # Creamos un numero aleatorio
            solution[i] = assign_slots
        
        else:
            solution[i] = remaining
            
    # Ahora necesitamos mezclar las estaciones, ya que es probable que las primeras estaciones tengan mas slots que las ultimas
    np.random.shuffle(solution)

    return solution.astype(int)

## Movement_operator

### Movement_operator Greedy

In [27]:
def greedy_movement_operator(actual_state, actual_solution, neighbours_limit, n_slots):
        """
        Operador de movimiento para el algoritmo greedy. Genera nuevos vecinos a partir de la solucion actual
        
        Parametros
            actual_state : Numpy ndarray
                Estado actual (estado inicial en nuestro problema)
            actual_solution : Numpy ndarray
                Solucion sobre la cual generar nuevos vecinos
            neighbours_limit : int
                Numero de vecinos maximos que se pueden generar. El valor maximo de 120 viene dado por ser el numero maximo de convinaciones posibles entre 16 estaciones
            n_slots : int
                Numero de slots a mover para generar una nueva solucion vecina
        Return
            Lista de soluciones vecinas
        """
        
        n = len(actual_solution) # Guardamos el tamanio de la solucion para hacer los bucles
        neighbours_list = np.array([]) # Lista de vecinos encontrados
        neighbours_n = 0 # Numero de vecinos encontrados
        
        # Recorremos todos los posibles vecinos (movimiento de n_slots entre todas las combinaciones de estaciones posibles)
        for i in range(1, n):
            for j in range(i, n): 
                # Buscamos todos los vecinos posibles que no superen nuestro limite de exploracion
                if neighbours_n < neighbours_limit:
                    # Ahora comprobamos si cambio de n_slots entra las estaciones 'i' y 'j' es posible realizar (la capacidad de una estacion no puede ser negativa)
                    if actual_solution[i] >= n_slots and (actual_solution[i]-n_slots) >= actual_state[i]:
                        neighbours_n += 1 # Aumentamos el numero de vecinos encontrados
                        
                        new_neighbour = actual_solution.copy() # Creamos una copia de la solucion actual para poder crear la vecina
                        new_neighbour[i] -= n_slots # Le quitamos n_slots a la estacion 'i'
                        new_neighbour[j] += n_slots # Le aniadimos los n_slots quitados de 'i' a 'j'
                        
                        if len(neighbours_list) == 0:
                            neighbours_list = np.array([new_neighbour])
                        else:
                            neighbours_list = np.append(neighbours_list, np.array([new_neighbour]), axis=0) # Aniadimos una nueva fila(axis=0) a la lista de vecinos. axis=0 para que la nueva solucion se aniada como fila
                            
        # Devolvemos el mejor vecino
        return neighbours_list


### Movement_operator común

In [28]:
def movement_operator_list(actual_state, actual_solution, n_slots, neighbours_limit, neighbours_map):
    """
    Operador de movimiento que nos da una lista de vecinos. Genera nuevos vecinos a partir de la solucion actual

    Parametros
        actual_state : Numpy ndarray
            Estado actual (estado inicial en nuestro problema)
        actual_solution : Numpy ndarray
            Solucion sobre la cual generar nuevos vecinos
        n_slots : int
            Numero de slots a mover para generar una nueva solucion vecina
        neighbours_limit : int
            Numero de vecinos validos maximo a generar. El valor maximo de 120 viene dado por ser el numero maximo de convinaciones posibles entre 16 estaciones
        neighbours_map : Numpy ndarray
            Lista que contiene todas las combinaciones posibles entre estaciones. Cada posicion de la lista tiene una sublistas con las indices de las estaciones a intercambiar
    Return
        Lista de soluciones vecinas
    """

    n_stations = len(actual_solution) # Guardamos el tamanio de la solucion para hacer los bucles

    # Tenemos que decidir un vecino por el que empezar a explorar. Este vecino sera escogido de forma aleatoria en el mapa de vecinos
    last_neighbours = len(neighbours_map) # Numero total de vecinos
    index = np.random.randint(0, last_neighbours) # Seleccionamos el primer vecino a explorar
    searched = 0 # Contador de total de vecinos visitados

    neighbours_list = np.array([]) # Lista de vecinos encontrados
    neighbours_n = 0 # Numero de vecinos encontrados

    # Exploramos todos los vecinos que podamos hasta llegar al limite de vecinos que podemos explorar (neighbours_limit), si se exploran todos los posibles salimos del bucle
    while neighbours_n < neighbours_limit and searched < last_neighbours:
        i = neighbours_map[index][0] # Estacion i
        j = neighbours_map[index][1] # Estacion j

        # Ahora comprobamos si el cambio de n_slots entra las estaciones 'i' y 'j' es posible realizar (la capacidad de una estacion no puede ser negativa)
        if actual_solution[i] >= n_slots and (actual_solution[i]-n_slots) >= actual_state[i]:
            neighbours_n += 1 # Aumentamos el numero de vecinos encontrados

            new_neighbour = actual_solution.copy() # Creamos una copia de la solucion actual para poder crear la vecina
            new_neighbour[i] -= n_slots # Le quitamos n_slots a la estacion 'i'
            new_neighbour[j] += n_slots # Le aniadimos los n_slots quitados de 'i' a 'j'

            if len(neighbours_list) == 0:
                neighbours_list = np.array([new_neighbour])
            else:
                neighbours_list = np.append(neighbours_list, np.array([new_neighbour]), axis=0) # Aniadimos una nueva fila(axis=0) a la lista de vecinos. axis=0 para que la nueva solucion se aniada como fila

        # Aumentomos la variable index para comprobar el siguiente vecino, comprobar volver al principio del mapa si llegamos al final del mismo
        index += 1
        if index == last_neighbours:
            index = 0 # Hacemos que siga explorando desde el principio del mapa

        # Aumentamos el numero tota de vecinos buscados
        searched += 1

    # Devolvemos la lista de vecinos encontrados
    return neighbours_list


In [ ]:
def movement_operator(actual_state, actual_solution, n_slots, neighbours_map, init_index, actual_index):
    """
    Operador de movimiento que nos devuelve un vecino. Usando el mapa de vecinos y la variable init_index nos devuelve el primer vecino valido. Ademas, junto al vecino
    nos devuelve el siguiente vecino a buscar en caso de que el vecino encontrado previamente no satisfada nuestros requisitos. 

    Parametros
        actual_state : Numpy ndarray
            Estado actual (estado inicial en nuestro problema)
        actual_solution : Numpy ndarray
            Solucion sobre la cual generar nuevos vecinos
        n_slots : int
            Numero de slots a mover para generar una nueva solucion vecina
        neighbours_map : Numpy ndarray
            Lista que contiene todas las combinaciones posibles entre estaciones. Cada posicion de la lista tiene una sublistas con las indices de las estaciones a intercambiar
        init_index : int
            Posicion inicial por donde se debe empezar a buscar vecinos en el mapa de vecinos
        actual_index : int
            Siguiente vecino a comprobar
    Return
        Devuelve una lista de dos posiciones. En la primera posicion se guarda el vecino encontrado y en la segunda posicion el siguiente vecino a buscar. En caso de que se haya explorado
        todos los vecinos posibles, se devolvera la siguiente lista: [None, -1]. Indicando que el espacio de busqueda ya ha sido explorado
    """

    n_stations = len(actual_solution) # Guardamos el tamanio de la solucion para hacer los bucles

    # Tenemos que decidir un vecino por el que empezar a explorar. Este vecino sera escogido de forma aleatoria en el mapa de vecinos
    last_neighbours = len(neighbours_map) # Numero total de vecinos
    actual_index = index # Seleccionamos el primer vecino a explorar
    neighbour_finded = False # Contador de total de vecinos visitados

    neighbours_n = 0 # Numero de vecinos encontrados
    
    # Exploramos todos los vecinos que podamos hasta llegar al limite de vecinos que podemos explorar (index - 1), si se exploran todos los posibles salimos del bucle
    while neighbour_finded == False and (init_index-1) != actual_index:
        i = neighbours_map[actual_index][0] # Estacion i
        j = neighbours_map[actual_index][1] # Estacion j

        # Ahora comprobamos si el cambio de n_slots entra las estaciones 'i' y 'j' es posible realizar (la capacidad de una estacion no puede ser negativa)
        if actual_solution[i] >= n_slots and (actual_solution[i]-n_slots) >= actual_state[i]:
            neighbour_finded = True # Hemos encontrado un vecino

            neighbour = actual_solution.copy() # Creamos una copia de la solucion actual para poder crear la vecina
            neighbour[i] -= n_slots # Le quitamos n_slots a la estacion 'i'
            neighbour[j] += n_slots # Le aniadimos los n_slots quitados de 'i' a 'j'

        # Aumentomos la variable index para comprobar el siguiente vecino, comprobar volver al principio del mapa si llegamos al final del mismo
        actual_index += 1
        if actual_index == last_neighbours:
            actual_index = 0 # Hacemos que siga explorando desde el principio del mapa

    if (init_index-1) == actual_index:
        neighbour = None
        actual_index = -1

    # Devolvemos la lista de vecinos encontrados
    return [neighbour, actual_index]

In [29]:
def create_neigbours_map(n_stations):
    """
    Crea una lista con todas las combinaciones posibles de estaciones. En dicha lista, se guardan los indices de cada estacion
    
    Parametros
        n_stations : int
            Numero de estaciones
    
    Return 
        Devuelve un Numpy ndarray con la siguiente estructura. Lista: [[estacion_i=1, estacion_j=2],[1,3],[1,4], ...]
    """
    neighbours_map = np.array([])
    for i in range(1, n_stations):
        for j in range(i, n_stations):
            if len(neighbours_map) != 0:
                neighbours_map = np.append(neighbours_map,np.array([[i,j]]), axis=0)
            else:
                neighbours_map = np.array([[i,j]])
                
    return neighbours_map

# Algoritmos

## Greedy

Para efectuar la comparativa de resultados entre los distintos algoritmos de búsqueda, se debe
implementar como algoritmo básico, un Greedy, es decir, siguiendo la heurística de asignar las
capacidades proporcionalmente al vector inicial de estado prestando atención a que la cantidad
total de plazas sea la indicada.

In [85]:
def greedy(max_slots, n_slots, neighbours_limit, init_state, move_list, index_m, kms_m, display):
    """
    Ejecuta el algoritmo Greedy para el problema de la practica 1

    Parametros
        max_slots : int
            Numero maximo de slots
        n_slots : int
            Numero de slots a mover entre una estacion y otra
        neighbours_limit : int 
            Numero maximo de vecinos a explorar
        init_state : Numpy ndarray
            Estado inicial de las estaciones
        move_list : Numpy ndarray
            Lista con todos los movimientos realizados en las estaciones (delta)
        index_m : Numpy ndarray
            Almacena las estaciones mas cercanas a cada estacion ordenadas de mas cercanas a mas lejanas. La fila indica la estacion de refencia, mientras que la columna como de cerca esta la estacion
        kms_m : Numpy ndarray
            Almacena la distancia que hay desde una estacion a otra. La disposicion de la informacion se basa en la variable index_m. Es decir, el valor de cada celda de esta matriz corresponde a la posicion homologa en index_m
        display : bool
            Si es igual a 'True' mostrara por pantalla todas las soluciones que vaya encontrando ademas de una comparativa final entre la primer y la ultima solucion encontrada. Dejar a 'False' para no mostrar nada
    Return
        Devuelve una lista con el vector solucion y el numero de kilometros recorridos por los usuarios en dicha solucion - [best_solution, best_tkms]
    """
    init_solution = generate_initial_greedy_solution(init_state, max_slots) # Generamos la solucion inicial
    init_tkms = evaluate(move_list, init_state, init_solution, index_m, kms_m) # Evaluamos nuestra solucion
    
    print(init_tkms)
    
    best_solution = init_solution.copy() # Mejor solucion
    best_tkms = init_tkms

    move_list = movements_to_list(deltas_df) # Creamos la lista de movimientos

    actual_solution = init_solution.copy()
    solution_finded = True # Variable de control, se establece a True para que entre al menos una vez en el bucle

    if display == True:
        print('Greedy init solution: {} - {} kms'.format(init_solution, best_tkms))

    # Debemos seguir buscando mientras se siga encontrando una mejor solucion
    while solution_finded == True: 
        solution_finded = False # Reseteamos la variable
        neigbours_list = greedy_movement_operator(init_state, actual_solution, neighbours_limit, n_slots) # Lista de vecinos de la solucion actual
        for neighbour in neigbours_list:
            actual_tkms = evaluate(move_list, init_state, neighbour, index_m, kms_m) # Evaluamos el vecino
            if best_tkms > actual_tkms:
                solution_finded = True 
                best_solution = neighbour.copy()
                best_tkms = actual_tkms
                if display == True:
                    print('New Greedy best solution: {} - {} kms '.format(best_solution, best_tkms))

    if display == True: 
        show_variation(init_solution, init_tkms, best_solution, best_tkms)

    return [best_solution, best_tkms]

## Busqueda aleatoria

El Algoritmo de Búsqueda Aleatoria (BA) consistirá en generar aleatoriamente una solución
en cada iteración debiéndose ejecutar 100 iteraciones con cada semilla devolviendo la mejor de
las iteraciones.

La búsqueda aleatoria completa debe ejecutarse 5 veces, cada vez con una semilla distinta (por
tanto, se deben anotar las 5 semillas que se utilizarán sistemáticamente), para el generador
aleatorio.

### Pseudocódigo del algoritmo
![title](img/random_search-pseudo.png)

In [31]:
def random_search(seed, max_slots, n_slots, init_state, move_list, index_m, kms_m, display):
    """
    Generamos una solucion de forma aleatoria

    Parametros
        seed : int
            Semilla con la que generar los numeros aleatorios
        max_slots : int
            Numero maximo de slots
        n_slots : int
            Numero de slots a mover entre una estacion y otra
        neighbours_limit : int 
            Numero maximo de vecinos a explorar
        init_state : Numpy ndarray
            Estado inicial de las estaciones
        move_list : Numpy ndarray
            Lista con todos los movimientos realizados en las estaciones (delta)
        index_m : Numpy ndarray
            Almacena las estaciones mas cercanas a cada estacion ordenadas de mas cercanas a mas lejanas. La fila indica la estacion de refencia, mientras que la columna como de cerca esta la estacion
        kms_m : Numpy ndarray
            Almacena la distancia que hay desde una estacion a otra. La disposicion de la informacion se basa en la variable index_m. Es decir, el valor de cada celda de esta matriz corresponde a la posicion homologa en index_m
        display : bool
            Si es igual a 'True' mostrara por pantalla todas las soluciones que vaya encontrando ademas de una comparativa final entre la primer y la ultima solucion encontrada. Dejar a 'False' para no mostrar nada
    Return
        Devuelve una lista con el vector solucion y el numero de kilometros recorridos por los usuarios en dicha solucion - [best_solution, best_tkms]
    """
    np.random.seed(seed) # Inicializamos la semilla
    
    # Creamos nuestra solucion inicial
    n_stations = len(init_state) # Numero de estaciones
    init_solution = generate_initial_solution(init_state, max_slots) 
    init_tkms = evaluate(move_list, init_state, init_solution, index_m, kms_m) # Evaluamos la solucion mejor solucion que ahora mismo es la inicial
    
    best_solution = init_solution.copy()
    best_tkms = init_tkms
        
    if display == True:
        print('BA init solution: {} - {} kms'.format(init_solution, best_tkms))
    
    # Ahora generamos 100 soluciones diferentes y nos quedamos con la mejor de ellas
    for i in range(0,100):
        actual_solution = generate_initial_solution(init_state, max_slots) # Generamos una nueva solucion
        actual_tkms = evaluate(move_list, init_state, actual_solution, index_m, kms_m) # Evaluamos la solucion actual
        if actual_tkms < best_tkms:
            best_solution = actual_solution.copy()
            best_tkms = actual_tkms
            if display == True:
                print('New BA best solution: {} - {} kms '.format(best_solution, best_tkms))
    
    if display == True:
        show_variation(init_solution, init_tkms, best_solution, best_tkms)
            
    return [best_solution, best_tkms]

## Búsqueda Local

Se implementará el esquema de **el primer mejor** vecino, según el Tema 1 de teoría.

Se partirá de una solución inicial aleatoria. Los algoritmos de búsqueda local tienen su propia condición de parada, pero adicionalmente, en prevención de tiempo excesivos en algún caso, se añadirá una condición de parada alternativa (OR) basada en el número de evaluaciones que esté realizando la búsqueda, es decir, el número de veces que se llame al cálculo de la función de coste. Este valor para la Búsqueda Local será de 3000 llamadas a la función de coste.

La búsqueda con cada algoritmo se debe ejecutar 5 veces con semilla distintas como en el caso de la Aleatoria.

### Pseudocódigo

![title](img/local_search_pseudo.png)

In [32]:
def local_search(seed, max_slots, n_slots, neighbours_limit, init_state, move_list, index_m, kms_m, display):
    """
    Devuelve una solucion usando el algoritmo de busqueda local. Se implementa el esquema de primero mejor vecino
        
    Parametros
        seed : int
            Semilla con la que generar los numeros aleatorios
        max_slots : int
            Numero maximo de slots
        n_slots : int
            Numero de slots a mover entre una estacion y otra
        neighbours_limit : int 
            Numero maximo de vecinos a explorar
        init_state : Numpy ndarray
            Estado inicial de las estaciones
        move_list : Numpy ndarray
            Lista con todos los movimientos realizados en las estaciones (delta)
        index_m : Numpy ndarray
            Almacena las estaciones mas cercanas a cada estacion ordenadas de mas cercanas a mas lejanas. La fila indica la estacion de refencia, mientras que la columna como de cerca esta la estacion
        kms_m : Numpy ndarray
            Almacena la distancia que hay desde una estacion a otra. La disposicion de la informacion se basa en la variable index_m. Es decir, el valor de cada celda de esta matriz corresponde a la posicion homologa en index_m
        display : bool
            Si es igual a 'True' mostrara por pantalla todas las soluciones que vaya encontrando ademas de una comparativa final entre la primer y la ultima solucion encontrada. Dejar a 'False' para no mostrar nada
    Return
        Devuelve una lista con el vector solucion y el numero de kilometros recorridos por los usuarios en dicha solucion - [best_solution, best_tkms]
    """
    # Inicializamos la semilla
    np.random.seed(seed) 
    
    # Creamos nuestra solucion inicial
    n_stations = len(init_state) # Numero de estaciones
    init_solution = generate_initial_solution(init_state, max_slots) 
    init_tkms = evaluate(move_list, init_state, init_solution, index_m, kms_m) # Evaluamos la solucion mejor solucion que ahora mismo es la inicial
    
    if display == True:
        print('LS init solution: {} - {} kms'.format(init_solution, init_tkms))
    
    # Necesitamos crear un mapa de vecinos para el operador de movimiento. Crearemos una lista donde cada posicion representa cada una de las 
    # posibles convinaciones de estaciones posibles. De esta forma, podremos seleccionar una de forma aleatoria y seguir explorando a partir de 
    # dicho vecino
    neighbours_map = create_neigbours_map(n_stations) # Mapa de vecinos
    
    # Inicializamos las variables
    actual_solution = init_solution.copy()
    actual_tkms = init_tkms
    best_neighbour = None
    best_tkms = None
    
    neighbour_tkms = None
    
    # Variables de control
    max_i = 3000 # Numero maximo de intentos a realizar
    i = 0 # Iteracion actual
    
    max_possible_neighbours = len(neighbours_map) # Numero maximo de vecinos posibles
    j = 0 
    neighbour_finded = True
    
    # Buscamos mientras no superemos el numero maximo de busquedas (max_calls) y siempre que encontremos una solucion mejor a la actual en la nueva iteracion
    while i < max_i and neighbour_finded == True:
        # Guardamos el mejor vecino encontrado
        best_neighbour = actual_solution
        best_neighbour_tkms = actual_tkms
        
        # Generamos los vecinos de la solucion actual
        j = 0 # Reseteamos el contador
        while j < max_possible_neighbours:
            neighbours_list = movement_operator(init_state, actual_solution, n_slots, neighbours_limit, neighbours_map) # Movement_operator devuelve una lista
            
            # Debemos comprobar que se ha encontrado un vecino candidato
            if len(neighbours_list) != 0:
                neighbour = neighbours_list[0].copy()
            else:
                break # Salimos del bucle
                
            neighbour_tkms = evaluate(move_list, init_state, neighbour, index_m, kms_m)
            
            # Si el vecino encontrado es mejor que nuestra mejor solucion
            if best_neighbour_tkms > neighbour_tkms:
                best_neighbour = neighbour.copy()
                best_neighbour_tkms = neighbour_tkms
                break # Estamos buscando el primer mejor vecino, por lo tanto, salimos del bucle
                
            # Si no se ha encontrado ningun vecino en la ultima comprobacion posible, salimos del bucle porque no podemos mejorar la solucion
            if j == max_possible_neighbours-1:
                neighbour_finded = False
                
            j += 1 # Nuevo vecino
        
        # Comprobamos si la solucion obtenida por el vecino es mejor que nuestra solucion actual
        if best_neighbour_tkms < actual_tkms: 
            # Guardamos al vecino como la nueva solucion actual y seguimos buscando
            actual_solution = best_neighbour.copy()
            actual_tkms = best_neighbour_tkms
            if display == True:
                print('New LS best solution: {} - {} kms '.format(actual_solution, actual_tkms))
            
        i += 1 # Nueva iteracion
        
    if display == True:
        show_variation(init_solution, init_tkms, actual_solution, actual_tkms)
       
    return [actual_solution, actual_tkms]        
                

## Enfriamiento simulado

Se ha de implementar el algoritmo de Enfriamiento Simulado (ES) con las siguientes
componentes:

* Esquema de enfriamiento: Se implementará el esquema de Cauchy, Tk = T0/(1 + k)
* Condición de enfriamiento L(T): Se enfriará la temperatura, y finalizará a la iteración actual, cuando se haya generado un número máximo de vecinos (independientemente de si han sido o no aceptados).
* Condición de parada: El algoritmo finalizará cuando se alcance un número máximo de iteraciones (enfriamientos).

Se calculará la fórmula inicial mediante la siguiente fórmula:
    
        T0 = (μ / -log(Φ)) * C(Si)

donde T0 es la temperatura inicial, C(Si) es el costo de la solución inicial y Φ[0,1] es la
probabilidad de aceptar una solución un µ por 1 peor que la inicial. En las ejecuciones se
considerar Φ=µ= 0,3.

El número de soluciones generadas en cada temperatura será L(T) = 20 y el número de
enfriamientos (iteraciones) será 80. Se debe repetir también 5 veces con distintas semillas, partiendo de una solución inicial
aleatoria.

![title](img/simulated_annealingV2_pseudo.png)

In [37]:
def simulated_annealing(seed, max_slots, n_slots, neighbours_limit, init_state, move_list, index_m, kms_m, display, iterations):
    # seed, max_slots, n_slots, neighbours_limit, init_state, move_list, index_m, kms_m, display
    """
    Devuelve una solucion usando el algoritmo de enfriamiento simulado
        
    Parametros
        seed : int
            Semilla con la que generar los numeros aleatorios
        max_slots : int
            Numero maximo de slots
        n_slots : int
            Numero de slots a mover entre una estacion y otra
        neighbours_limit : int 
            Numero maximo de vecinos a explorar para cada nivel de temperatura
        init_state : Numpy ndarray
            Estado inicial de las estaciones
        move_list : Numpy ndarray
            Lista con todos los movimientos realizados en las estaciones (delta)
        index_m : Numpy ndarray
            Almacena las estaciones mas cercanas a cada estacion ordenadas de mas cercanas a mas lejanas. La fila indica la estacion de refencia, mientras que la columna como de cerca esta la estacion
        kms_m : Numpy ndarray
            Almacena la distancia que hay desde una estacion a otra. La disposicion de la informacion se basa en la variable index_m. Es decir, el valor de cada celda de esta matriz corresponde a la posicion homologa en index_m
        display : bool
            Si es igual a 'True' mostrara por pantalla todas las soluciones que vaya encontrando ademas de una comparativa final entre la primer y la ultima solucion encontrada. Dejar a 'False' para no mostrar nada
        iterations : int
            Cantidad de veces a reducir la temperatura. Una vez que se realicen todas las reducciones el algoritmo termina
            
    Return
        Devuelve una lista con el vector solucion y el numero de kilometros recorridos por los usuarios en dicha solucion - [best_solution, best_tkms]
    """
    # Inicializamos la semilla
    np.random.seed(seed) 
    
    # Necesitamos crear un mapa de vecinos para el operador de movimiento. Crearemos una lista donde cada posicion representa cada una de las 
    # posibles convinaciones de estaciones posibles. De esta forma, podremos seleccionar una de forma aleatoria y seguir explorando a partir de 
    # dicho vecino
    n_stations = len(init_state)
    neighbours_map = create_neigbours_map(n_stations) # Mapa de vecinos
    
    mu = 0.3 
    phi = 0.3 # Probabilidad de aceptar una solucion
    init_solution = generate_initial_solution(init_state, max_slots)
    init_tkms = evaluate(move_list, init_state, init_solution, index_m, kms_m)
    
    init_t = (mu / -np.log10(phi)) * init_tkms # Temperatura inicial
    
    t = init_t # Temperatura actual
    k = 1.380649 * pow(10,-23) # Constante de Boltzmann / pow(base, numero a elevar)
    tk = init_t / (1 + k) # Mecanismo de enfriamiento, esquema de Cauchy
    L = neighbours_limit # Numero de vecinos generados para cada valor de temperatura 
    
    if display == True:
        print('SA init solution: {} - {} kms'.format(init_solution, init_tkms))
    
    actual_solution = init_solution.copy()
    actual_tkms = init_tkms
    
    # En el algoritmo original tenemos una temperatura y vamos reduciendo su valor. Salimos cuando la temperatura haya disminuido lo suficiente: t >= final_t
    # Nosotros estableceremos un numero de iteraciones maximo
    i = 0
    while i < iterations:
        if display == True:
            print('Iteracion - {} | Temperatura - {}'.format(1+i, t))
            
        for j in range(0, L):
            neighbours_list = movement_operator(init_state, actual_solution, n_slots, 1, neighbours_map) # Movement_operator devuelve una lista, con el valor 1 nos devuelve 1 vecino
            
            # Debemos comprobar que se ha encontrado un vecino candidato
            if len(neighbours_list) != 0:
                neighbour = neighbours_list[0].copy()
            
                neighbour_tkms = evaluate(move_list, init_state, neighbour, index_m, kms_m)
                delta = neighbour_tkms - actual_tkms # Calculo de la diferencia de costos

                # Aplicamos el criterio de aceptacion
                # Tenemmos que usar un numero uniforme, un valor aleatorio entre 0 y 1
                valor_uniforme = np.random.rand()
                if (valor_uniforme < np.exp(-delta / t)) or (delta < 0):
                    actual_solution = neighbour.copy()
                    actual_tkms = neighbour_tkms
                    #if display == True:
                    #    print('SA init solution: {} - {} kms'.format(actual_solution, actual_tkms))
            else:
                break # Si no se ha encontrado ningun vecino valido debemos dejar de generar vecinos
                
        t = init_t / (i + k) # Reducimos la temperatura
        k += 1 # Elevamos el valor de k para que en la siguiente iteracion se vuelve a reducir la temperatura
        i += 1
    
    if display == True:
        show_variation(init_solution, init_tkms, actual_solution, actual_tkms)
                
    return [actual_solution, actual_tkms]
    

## Búsqueda Tabú

Se implementará la versión de BT utilizando una lista de movimientos tabú y tres estrategias
de reinicialización.

Sus principales características son:

* Estrategia de selección de vecino: Consistirá en examinar 40 vecinos para coger el mejor de acuerdo a los criterios tabú
* Codificación tabú: codificación de valores en cada estación. Cuantas veces toma un valor una estación.
* Selección de estrategias de reinicialización: La probabilidad de escoger la reinicialización construyendo una solución inicial aleatoria es 0,25, la de usar la memoria a largo plazo al generar una nueva solución greedy es 0,5, y la de utilizar la reinicialización desde la mejor solución obtenida es 0,25.
* La solución greedy debe generar soluciones con mayor probabilidad para los valores que menos veces se han producido en cada estación. Para cada estación se deberá calcular las inversas de los valores acumulados y luego normalizar para tener valores entre 0-1 correspondiente a su probabilidad. Se tirará un dado y se elige el primer valor que supera dicho valor añadiéndose a la solución greedy

In [ ]:
def tabu_search(seed, max_slots, n_slots, neighbours_limit, init_state, move_list, index_m, kms_m, display, iterations):
    """
    
    """
    # Inicializamos la semilla
    np.random.seed(seed) 
    
    # Necesitamos crear un mapa de vecinos para el operador de movimiento. Crearemos una lista donde cada posicion representa cada una de las 
    # posibles convinaciones de estaciones posibles. De esta forma, podremos seleccionar una de forma aleatoria y seguir explorando a partir de 
    # dicho vecino
    n_stations = len(init_state)
    neighbours_map = create_neigbours_map(n_stations) # Mapa de vecinos
    
    frequency_matrix = np.zeros((n_stations, n_stations)) # Matriz de frecuencias donde se almacenara el numero de veces en las que se ha visitado un vecino
    tabu_max_size = 4 # Tamanio inicial de cada lista tabu
    tabu_list = np.array([])
    
    # Creamos las variables
    init_solution = generate_initial_solution(init_state, max_slots) 
    init_tkms = evaluate(move_list, init_state, init_solution, index_m, kms_m)
    actual_solution = init_solution.copy()
    actual_tkms = init_tkms
    best_solution = actual_solution.copy()
    best_tkms = actual_tkms
    
    
    # Estrategia de seleccion: consistira en examinar 40 vecinos para coger el mejor de los criterios tabu
    for i in range(0,iterations):        
        # Debemos reiniciar la lista tabu cada 10 iteraciones. Ademas, debemos aumentar la lista tabu o aumentarla en funcion de un random del 50%
        if (i % 10) == 0 and i != 0:
            rand = np.random.rand() # Creamos un numero aleatorio
            if rand < 0.5: # Si rand es mejor que 0.5 reducimos el tamanio de la lista
                tabu_max_size -= tabu_max_size
            else: # En caso contrario aumentamos su tamanio
                tabu_max_size += tabu_max_size
                
            tabu_list = np.array([]) # Reinicializamos la lista tabu
            
            '''
            Seleccionamos una estrategia de de reinicializacion: La probabilidad de escoger la reinicializacion contruyendo una solucion inicial aleatoria es 0,25,
            la de usar la memoria a largo plazo al generar una nueva solucion greedy es 0,5, y la de utilizar la reinicializacion desde la mejor solucion es 0,25
            '''
            rand = np.random.rand() # Creamos un nuevo numero aleatorio
            if rand < 0.25: # Solucion aleatoria
                actual_solution = generate_initial_solution(init_state, max_slots) # Creamos una nueva solucion aleatoria
                actual_tkms = evaluate(move_list, init_state, actual_solution, index_m, kms_m)
            elif rand < 0.5: # Mejor solucion
                actual_solution = best_solution# Seleccionamos la mejor solucion
                actual_tkms = best_tkms
            else: # Solucion greedy
                # Mejor solucion hasta ahora encontrada?
            


            '''

            '''

# Test

## Greedy test

In [86]:
import numpy as np
import pandas as pd
import time

index_df = pd.read_csv('./bicicletas/cercanas_indices.csv')
kms_df = pd.read_csv('./bicicletas/cercanas_kms.csv')
deltas_df = pd.read_csv('./bicicletas/deltas_5m_double.csv')

index_m = index_df.to_numpy() # index matrix
kms_m = kms_df.to_numpy() # kms matrix
move_list = movements_to_list(deltas_df) # Lista de movimientos
init_state = deltas_df.iloc[0].to_numpy() # Estado inicial

seed = 1 # Semilla inicial
max_slots = 220 # Numero de slots maximos a repartir
n_slots = 3 # Numero de slots a mover en cada movimiento
neighbours_limit = 120 # Numero maximo de vecinos a explorar

display = False

start = time.time()
greedy_solution, greedy_tkms = greedy(max_slots, n_slots, neighbours_limit, init_state, move_list, index_m, kms_m, display)
finish = time.time()

print('Greedy best solution: {} - {} kms '.format(greedy_solution, greedy_tkms))
print('Tiempo empleado: {} s'.format(finish-start))

[ 7  9 18  8 11 18 11 12  8 13 13 24 11 18 20 19]
593.4323852090002
Greedy best solution: [ 7  9 15  8 14 18 11 12  8 13 13 24 11 18 20 19] - 525.9898883948373 kms 
Tiempo empleado: 0.973595380783081 s


A marina el greedy le da 576kms

## Busqueda aleatoria Test

In [79]:
import numpy as np
import pandas as pd
import time

index_df = pd.read_csv('./bicicletas/cercanas_indices.csv')
kms_df = pd.read_csv('./bicicletas/cercanas_kms.csv')
deltas_df = pd.read_csv('./bicicletas/deltas_5m_double.csv')

index_m = index_df.to_numpy() # index matrix
kms_m = kms_df.to_numpy() # kms matrix
move_list = movements_to_list(deltas_df) # Lista de movimientos
init_state = deltas_df.iloc[0].to_numpy() # Estado inicial

seeds = np.array([9,202,357,598,745]) # Semilla inicial
max_slots = 220 # Numero de slots maximos a repartir
n_slots = 2 # Numero de slots a mover en cada movimiento
neighbours_limit = 120 # Numero maximo de vecinos a explorar

init_solution = None # Inicializamos la primera solucion a null
init_tkms = None
ba_solution = None # Inicializamos la mejor solucion a null
ba_tkms = float('inf') # Igualamos la mejor distancia recorrida a infinito 

display = False # Muestra el progreso del algoritmo

start = time.time()
for i in seeds:
    if display == True:
        print('Semilla - ', i)
        
    actual_solution, actual_tkms = random_search(i, max_slots, n_slots, init_state, move_list, index_m, kms_m, display)
    if ba_tkms != float('inf'): # Si se ha asignado una solucion previamente, no valdra infinito
        if ba_tkms > actual_tkms:
            ba_solution = actual_solution.copy()
            ba_tkms = actual_tkms
            
    else: # Si no se ha asignado una solucion todavia
        init_solution = actual_solution.copy()
        init_tkms = actual_tkms
        ba_solution = actual_solution.copy()
        ba_tkms = actual_tkms

finish = time.time()        
print('BA best solution: {} - {} kms '.format(ba_solution, ba_tkms))
print('Tiempo empleado: {} s'.format(finish-start))

BA best solution: [15 11 14 14 14 14 11 10 11 11 18 19 11 15 17 15] - 391.20684351237855 kms 
Tiempo empleado: 7.382551193237305 s


## Busqueda local Test

In [80]:
import numpy as np
import pandas as pd
import time

index_df = pd.read_csv('./bicicletas/cercanas_indices.csv')
kms_df = pd.read_csv('./bicicletas/cercanas_kms.csv')
deltas_df = pd.read_csv('./bicicletas/deltas_5m_double.csv')

index_m = index_df.to_numpy() # index matrix
kms_m = kms_df.to_numpy() # kms matrix
move_list = movements_to_list(deltas_df) # Lista de movimientos
init_state = deltas_df.iloc[0].to_numpy() # Estado inicial

seeds = np.array([9,202,357,598,745]) # Semilla inicial
max_slots = 220 # Numero de slots maximos a repartir
n_slots = 3 # Numero de slots a mover en cada movimiento
neighbours_limit = 1 # Numero maximo de vecinos a explorar, devuelveme el primero valido

init_solution = None # Inicializamos la primera solucion a null
init_tkms = None
ls_solution = None # Inicializamos la mejor solucion a null
ls_tkms = float('inf') # Igualamos la mejor distancia recorrida a infinito 

solution_list = np.array([])
tkms_list = np.array([])

display = False # Muestra el progreso del algoritmo

start = time.time()
for i in seeds:
    if display == True:
        print('Semilla - ', i)
        
    actual_solution, actual_tkms = local_search(i, max_slots, n_slots, neighbours_limit, init_state, move_list, index_m, kms_m, display)
    # Aniadimos la solucion a la lista de soluciones
    if len(solution_list) != 0:
        solution_list = np.append(solution_list, np.array([[actual_solution]]), axis=0)
        tkms_list = np.append(tkms_list, np.array([actual_tkms]))
    else:
        solution_list = np.array([[actual_solution]])
        tkms_list = np.array([actual_tkms])
        
    if ls_tkms != float('inf'): # Si se ha asignado una solucion previamente, no valdra infinito
        if ls_tkms > actual_tkms:
            ls_solution = actual_solution.copy()
            ls_tkms = actual_tkms
            
    else: # Si no se ha asignado una solucion todavia
        init_solution = actual_solution.copy()
        init_tkms = actual_tkms
        ls_solution = actual_solution.copy()
        ls_tkms = actual_tkms
        
finish = time.time()
        
if display == True:
    print('Resultados de cada semilla')
    for i in range(0,len(seeds)):
        print('Semilla {} - Solucion: {} - {} km'.format(seeds[i], solution_list[i], tkms_list[i]))
    print('')
    
print('LS best solution: {} - {} km'.format(ls_solution, ls_tkms))
print('Tiempo empleado: {} s'.format(finish-start))

LS best solution: [12 12 14 12 14 14 10 10 11 14 15 22  8 14 17 21] - 369.3676697277217 km
Tiempo empleado: 4.022722244262695 s


## Enfriamiento simulado Test

In [82]:
import numpy as np
import pandas as pd
import time

index_df = pd.read_csv('./bicicletas/cercanas_indices.csv')
kms_df = pd.read_csv('./bicicletas/cercanas_kms.csv')
deltas_df = pd.read_csv('./bicicletas/deltas_5m_double.csv')

index_m = index_df.to_numpy() # index matrix
kms_m = kms_df.to_numpy() # kms matrix
move_list = movements_to_list(deltas_df) # Lista de movimientos
init_state = deltas_df.iloc[0].to_numpy() # Estado inicial

seeds = np.array([9,202,357,598,745]) # Semilla inicial
max_slots = 220 # Numero de slots maximos a repartir
n_slots = 2 # Numero de slots a mover en cada movimiento
neighbours_limit = 20 # Numero maximo de vecinos a explorar, devuelveme el primero valido
iterations = 80 # Numero de iteraciones a realizar

init_solution = None # Inicializamos la primera solucion a null
init_tkms = None
sa_solution = None # Inicializamos la mejor solucion a null
sa_tkms = float('inf') # Igualamos la mejor distancia recorrida a infinito 

solution_list = np.array([])
tkms_list = np.array([])

display = False # Muestra el progreso del algoritmo

start = time.time()
for i in seeds:
    if display == True:
        print('Semilla - ', i)
        
    actual_solution, actual_tkms = simulated_annealing(i, max_slots, n_slots, neighbours_limit, init_state, move_list, index_m, kms_m, display, iterations)
    # Aniadimos la solucion a la lista de soluciones
    if len(solution_list) != 0:
        solution_list = np.append(solution_list, np.array([[actual_solution]]), axis=0)
        tkms_list = np.append(tkms_list, np.array([actual_tkms]))
    else:
        solution_list = np.array([[actual_solution]])
        tkms_list = np.array([actual_tkms])
        
    if sa_tkms != float('inf'): # Si se ha asignado una solucion previamente, no valdra infinito
        if sa_tkms > actual_tkms:
            sa_solution = actual_solution.copy()
            sa_tkms = actual_tkms
            
    else: # Si no se ha asignado una solucion todavia
        init_solution = actual_solution.copy()
        init_tkms = actual_tkms
        sa_solution = actual_solution.copy()
        sa_tkms = actual_tkms
        
finish = time.time()

if display == True:
    print('Resultados de cada semilla')
    for i in range(0,len(seeds)):
        print('Semilla {} - Solucion: {} - {} km'.format(seeds[i], solution_list[i], tkms_list[i]))
    print('')    

print('SA best solution: {} - {} km'.format(sa_solution, sa_tkms))
print('Tiempo empleado: {} s'.format(finish-start))

SA best solution: [13  8 13  7  9 14  8 10  6 10 11 21 13 13 21 43] - 622.674666424299 km
Tiempo empleado: 39.47964644432068 s


## COMO MEDIR UN ALGORITMO

In [22]:
import time
start = time.time()
# Codigo a medir
finish = time.time()

In [44]:
a = np.array([[0,1,2,3,4],[10,11,12,13,14]])
a * 2

array([[ 0,  2,  4,  6,  8],
       [20, 22, 24, 26, 28]])

In [74]:

m = deltas_df.to_numpy()[1:]
m = m*2
f = deltas_df.to_numpy()[0]
m = np.append([f],m, axis=0)

#d = pd.DataFrame(m, columns = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]) 
deltas_df.head()
new_d = pd.DataFrame(m)
new_d.head()
new_d.to_csv('./bicicletas/deltas_5m_double.csv')